In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt
from pathlib import Path
from data_functions import  create_causal_data
from multi_outcomes import multi_outcomes_data


#### Adjust pandas Outcput

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [4]:
%%time
canary_p = '../../temp/CANARY.p'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pd.read_pickle(canary_p)
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 26.9 s, sys: 9.18 s, total: 36.1 s
Wall time: 19.2 s


In [5]:
pmpm.head(2)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2570122               95      40392        0             0                  1       0          1                    0             0          0              1          0           0       1                 0            0                  0                  1                     0        1                   0              1                        0                  0            0                0         12                      0            0                0            1                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2570122               94      19837        0             0                  0       0          1                    0             1          0              0          0           0       1                 0            0                  0                  0                     0        1                   0              0                        0                  0            0                0          7                      0            0                0            1                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR MBR_GNDR     MYR  
0            0                   0        0                 0                       1               0                0          0           0                   0                 0        M  202002  
1            0                   0        0                 0                       0               0                0          0           0                   0                 0        M  201810

#### An inspection of categories registered in biometrics data

In [6]:
%%time
pd.DataFrame(biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)).T

CPU times: user 19.1 s, sys: 5.1 s, total: 24.2 s
Wall time: 24.2 s


BIOMETRIC_CATEGORY     A1C       BMI    BNP  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  DCSI-SERUM  Ferritin    Height  Hematocrit  Hemoglobin    Iron  NT_proBNP     PSA  Potassium  Troponin    Weight    eGFR
0                   790853  10034183  42552                     4902004                    5238991            1305880     628758              81297     2734084    219613  11808045     2574412     2919909  197878       9258  310283    2666981     94030  14000267  907656

## Steps in creating causal data
``` python
%%time
from data_functions import  biometrics_f, pmpm_f, canary_f, merge_filter_separate, control_outcomes, treated_outcomes, combine_treated_controls
biometrics = biometrics_f(biometrics, 'BMI', True)
pmpm = pmpm_f(pmpm, True, 12)
canary = canary_f(canary)
merged, treated, controls, canary = merge_filter_separate(biometrics, pmpm, canary, merge_how = 'inner')
controls_final = control_outcomes(controls, max_days_apart = 180, min_days_apart = 120)
treated_final, treated_help = treated_outcomes(treated, max_days_apart = 180, min_days_apart = 120)
df = combine_treated_controls(treated_final, control_finals)
```

## Multi Outcomes Data

In [7]:
%%time
biometrics_list = ['BMI', 'A1C', 'Blood Pressure (Diastolic)', 'Blood Pressure (Systolic)', 'Cholesterol (LDL)', 'Weight']
data = multi_outcomes_data(pmpm.copy(), biometrics.copy(), canary.copy(), biometrics_list = biometrics_list, kwargs = None)

A total of 6 iterations will be run


/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls['OUTCOME'] = outcomes
/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  controls['OUTCOME2'] = outcome2
/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Iteration 0 of 6
Creating the data for biometrics with name: BMI


/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:461: RuntimeWarning: divide by zero encountered in true_divide
  outcome2 = ((res_value[1:]-res_value[0:-1])/res_value[0:-1])
/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:461: RuntimeWarning: invalid value encountered in true_divide
  outcome2 = ((res_value[1:]-res_value[0:-1])/res_value[0:-1])


Iteration 1 of 6
Creating the data for biometrics with name: A1C
Iteration 2 of 6
Creating the data for biometrics with name: Blood Pressure (Diastolic)
Iteration 3 of 6
Creating the data for biometrics with name: Blood Pressure (Systolic)
Iteration 4 of 6
Creating the data for biometrics with name: Cholesterol (LDL)
Iteration 5 of 6
Creating the data for biometrics with name: Weight
CPU times: user 9min 51s, sys: 1min 21s, total: 11min 13s
Wall time: 11min 13s


In [10]:
for df in data:
    print(df.shape, df.TREATED.sum())

(112716, 80) 371
(29173, 80) 46
(110937, 80) 260
(117614, 80) 336
(26831, 80) 90
(119016, 80) 386


In [12]:
df1, df2 = data[0], data[1]

In [14]:
df1.TREATED.sum(), df2.TREATED.sum()

(371, 46)

In [15]:
df3 = pd.merge(df1, df2, how = 'outer', on = ['PERS_ID'], )

In [16]:
df3

AGE_AT_MIDMONTH_x  ALLOW_AMT_x BIOMETRIC_CATEGORY_x  CC_ADHD_x  CC_ALZHEIMER_x  CC_ALZHEIMER_PLUS_x  CC_AMI_x  CC_ANEMIA_x  CC_ANXIETY_DISORDER_x  CC_ARTHRITIS_x  CC_ASTHMA_x  CC_ATRIAL_FIB_x  CC_AUTISM_x  CC_BIPOLAR_x  CC_BPH_x  CC_BREAST_CANCER_x  CC_CATARACT_x  CC_CEREBRAL_PALSY_x  CC_CHRONIC_KIDNEY_x  CC_COLORECTAL_CANCER_x  CC_COPD_x  CC_CYSTIC_FIBROSIS_x  CC_DEPRESSION_x  CC_DEPRESSIVE_DISORDERS_x  CC_DEVELOP_DELAYS_x  CC_DIABETES_x  CC_DISAB_DX_CNT_x  CC_DX_CNT_x  CC_ENDOMETRIAL_CANCER_x  CC_EPILEPSY_x  CC_FIBROMYALGIA_x  CC_GLAUCOMA_x  CC_HEARING_IMPAIR_x  CC_HEART_FAILURE_x  CC_HEPATITIS_A_x  CC_HEPATITIS_B_ACUTE_x  CC_HEPATITIS_B_CHRONIC_x  CC_HEPATITIS_C_ACUTE_x  CC_HEPATITIS_C_CHRONIC_x  CC_HEPATITIS_C_UNSPECIFIED_x  CC_HEPATITIS_D_x  CC_HEPATITIS_E_x  CC_HEPATITIS_GEN_x  CC_HIP_FRACTURE_x  CC_HYPERLIPIDEMIA_x  CC_HYPERTENSION_x  CC_HYPOTHYROID_x  CC_INT_DISAB_x  CC_ISCHEMIC_x  CC_LEARN_DISAB_x  CC_LEUKEMIAS_x  CC_LIVER_x  CC_LUNG_CANCER_x  CC_MIGRAINE_x  \
0                    81.0        414.0                  BMI        0.0             0.0                  0.0       0.0          0.0                    1.0             0.0          0.0              0.0          0.0           0.0       0.0                 0.0            0.0                  0.0                  0.0                     0.0        0.0                   0.0              0.0                        0.0                  0.0            0.0                1.0          2.0                      0.0            0.0                0.0            0.0                  1.0                 0.0               0.0                     0.0                       0.0                     0.0                       0.0                           0.0               0.0               0.0                 0.0                0.0                  0.0                1.0               0.0             0.0            0.0               0.0             0.0         0.0               0.0            0.0   
1                    82.0        101.0                  BMI        0.0             0.0                  0.0       0.0          0.0                    1.0             0.0          0.0              0.0          0.0           0.0       0.0                 0.0            0.0                  0.0                  0.0                     0.0        0.0                   0.0              0.0                        0.0                  0.0            0.0                0.0          3.0                      0.0            0.0                0.0            0.0                  0.0                 0.0               0.0                     0.0                       0.0                     0.0                       0.0                           0.0               0.0               0.0                 0.0                0.0                  1.0                1.0               0.0             0.0            0.0               0.0             0.0         0.0               0.0            0.0   
2                    78.0      13007.0                  BMI        0.0             0.0                  0.0       0.0          0.0                    0.0             1.0          0.0              0.0          0.0           1.0       0.0                 0.0            1.0                  0.0                  0.0                     0.0        0.0                   0.0              1.0                        0.0                  0.0            0.0                0.0          8.0                      0.0            0.0                0.0            0.0                  0.0                 0.0               0.0                     0.0                       0.0                     0.0                       0.0                           0.0               0.0               0.0                 0.0                0.0                  1.0                1.0               0.0             0.0            1.0               0.0             1.0         0.0               0.0            0.0   
3                    76.0        898.0  

In [11]:
data[0].head()

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0               81        414                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          2                      0            0                0            0                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
1               82        101                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
2               78      13007                BMI        0             0                  0       0          0                    0             1          0              0          0           1       0                 0            1                  0                  0                     0        0                   0              1                        0                  0            0                0          8                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             1         0               0            0                   0                      0                      0           0                0              0   
3               76        898                BM

In [9]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df1.to_pickle('../../matching_data/data.p')


In [10]:
df1.BIOMETRIC_CATEGORY.value_counts(), df2.BIOMETRIC_CATEGORY.value_counts()

(BMI    112855
 Name: BIOMETRIC_CATEGORY, dtype: int64, A1C    29191
 Name: BIOMETRIC_CATEGORY, dtype: int64)

In [11]:
df1.TREATED.value_counts(), df2.TREATED.value_counts()

(0    112345
 1       973
 Name: TREATED, dtype: int64, 0    29127
 1      973
 Name: TREATED, dtype: int64)

### <span style="color:red">Attention </span>
biometrics have datapoints from 1920, 1940... and one datapoint from the future 2179


### DoWhy demonstration on the data(install dowhy with)
```pip3 install dowhy --user```

In [12]:
import dowhy
from dowhy import CausalModel

import logging
logging.getLogger("dowhy").setLevel(logging.WARNING)
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

### Specify Common Causes Names

In [15]:
common_causes = np.setdiff1d(df1.columns, ['BIOMETRIC_CATEGORY', 'OUTCOME', 'OUTCOME2', 'OUTCOME3'
                                          'PERS_ID', 'RESULT_VALUE', 'TREATED', 'RESULT_VALUE2'])

df1.TREATED = df1.TREATED.astype(bool)
df2.TREATED = df2.TREATED.astype(bool)


### Create data dictionary for causal modeling

In [21]:
data1 = {'df':df1, 
        'common_causes_names':list(common_causes), 
        'outcome1':'OUTCOME', 
        'outcome2':'OUTCOME1',
        'outcome3':'OUTCOME3',
        'treatment_name':'TREATED'}

data2 = {'df':df2, 
        'common_causes_names':list(common_causes), 
        'outcome1':'OUTCOME', 
        'outcome2':'OUTCOME1',
        'outcome3':'OUTCOME3',
        'treatment_name':'TREATED'}

In [22]:
def causal_analysis(data, outcome_name,  method = "backdoor.propensity_score_stratification"):
    
    #define causal model
    model = CausalModel(
            data=data['df'],
            treatment=data['treatment_name'],
            outcome=data[outcome_name],
            common_causes=data['common_causes_names'],
            )
    
    #check if it is possible the estimand to be estimated
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    
    #calculate propensities
    propensity_match_estimate = model.estimate_effect(identified_estimand,
                            method_name = method,
                            test_significance = False,
                            confidence_intervals = False,
                            target_units="att")
    
    return propensity_match_estimate



In [23]:
m1 = causal_analysis(data1, 'outcome3')
m2 = causal_analysis(data2, 'outcome3')

/usr/local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### BMI RESULTS

In [24]:
m1.value

INFO:matplotlib.font_manager:generated new fontManager


### A1C RESULTS

In [25]:
m2.value